In [1]:
%matplotlib inline
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

## Feature Engineer Members Data

In [2]:
members = pd.read_csv('Data/members.csv', parse_dates=['registration_init_time','expiration_date'])

In [3]:
members.head(5)

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,2011-08-20,2017-09-20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,2015-06-28,2017-06-22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,2016-04-11,2017-07-12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,2015-09-06,2015-09-07
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,2017-01-26,2017-06-13


### Identify null count per member feature

In [4]:
for col in members.columns:
    print(col + ": ", members[col].isnull().sum())

msno:  0
city:  0
bd:  0
gender:  19902
registered_via:  0
registration_init_time:  0
expiration_date:  0


### Identify 0 count per members column

In [5]:
for col in members.columns:
    print(col + ": ", members[col][members[col] == 0].count())

msno:  0
city:  0
bd:  19932
gender:  0
registered_via:  0
registration_init_time:  0
expiration_date:  0


### Identify Members data types

In [6]:
for item in members.columns:
    print(item + ": " + str(members[item].dtype))

msno: object
city: int64
bd: int64
gender: object
registered_via: int64
registration_init_time: datetime64[ns]
expiration_date: datetime64[ns]


### Extract components of registration and expiration date

In [46]:
members['reg_day'] = members['registration_init_time'].dt.day
members['reg_month'] = members['registration_init_time'].dt.month
members['reg_year'] = members['registration_init_time'].dt.year

members['exp_day'] = members['expiration_date'].dt.day
members['exp_month'] = members['expiration_date'].dt.month
members['exp_year'] = members['expiration_date'].dt.year

In [47]:
members = members.drop(columns = ['registration_init_time', 'expiration_date'])

In [48]:
members.head()

,msno,city,bd,gender,registered_via,reg_day,reg_month,reg_year,exp_day,exp_month,exp_year
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20,8,2011,20,9,2017
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,28,6,2015,22,6,2017
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,11,4,2016,12,7,2017
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,6,9,2015,7,9,2015
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,26,1,2017,13,6,2017


### Fill NA for missing gender values

In [ ]:
members['gender'].fillna('no_gender', inplace=True)

### Generate dummy variables for gender

In [ ]:
members_new = pd.get_dummies(members, columns = ['gender'], drop_first = True)

In [ ]:
members_new.head()

# Feature Engineer Songs Data

In [7]:
songs = pd.read_csv('Data/songs.csv')

In [8]:
for col in songs.columns:
    print(col + ": ", songs[col].isnull().sum())

song_id:  0
song_length:  0
genre_ids:  94116
artist_name:  0
composer:  1071354
lyricist:  1945268
language:  1


In [9]:
for col in songs.columns:
    print(col + ": ", songs[col][songs[col] == 0].count())

song_id:  0
song_length:  0
genre_ids:  0
artist_name:  0
composer:  0
lyricist:  0
language:  0


In [10]:
for song in songs.columns:
    print(song + ": " + str(songs[song].dtype))

song_id: object
song_length: int64
genre_ids: object
artist_name: object
composer: object
lyricist: object
language: float64


In [11]:
songs['genre_ids'].fillna('no_genre', inplace=True)
songs['artist_name'].fillna('no_artist', inplace=True)
songs['composer'].fillna('no_composer', inplace=True)
songs['lyricist'].fillna('no_lyricist', inplace=True)

### Keep first of multiple values seperated by "|" for categorical values

### Replace composer column

In [23]:
composers = []
for composer in songs['composer']:
    result = re.split('\|+\W+', composer) # split using "|"
    composers.append(result[0]) # keep only the first for simplicity

In [24]:
songs = songs.drop(columns = ['composer'])
songs['composers'] = composers

### Replace Artist column

In [25]:
artists=[]
for artist in songs['artist_name']:
    result = re.split('\|+\W+', artist) # split using " | "
    artists.append(result[0]) # keep only the first for simplicity

In [26]:
songs = songs.drop(columns = ['artist_name'])
songs['artist'] = artists

### Replace Lyricist column

In [27]:
lyricists=[]
for lyricist in songs['lyricist']:
    result = re.split('\|+\W+', lyricist) # split using " | "
    lyricists.append(result[0]) # keep only the first for simplicity

In [28]:
songs = songs.drop(columns = ['lyricist'])
songs['lyricists'] = lyricists

In [29]:
songs.head()

,song_id,song_length,genre_ids,language,composers,artist,lyricists
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,3.0,董貞,張信哲 (Jeff Chang),何啟弘
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,31.0,TEDDY,BLACKPINK,TEDDY
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,31.0,no_composer,SUPER JUNIOR,no_lyricist
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,3.0,湯小康,S.H.E,徐世珍
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,52.0,Traditional,貴族精選,Traditional


### Drop handful of errorenous records

In [35]:
temp = songs[songs['lyricists'] == 'Robert Andrew Hyman']

In [36]:
temp

,song_id,song_length,genre_ids,language,composers,artist,lyricists
492446,8i5p+Fr3y2Cd5rUr931/y4NX4TrSaUJs28JAV2/pcjk=,207360,465,52.0,Robert Andrew Hyman,Janine Marx,Robert Andrew Hyman
771082,Ju0VGkjWeBUZCd7r5Az2hUImhMoWxWLUicOedsmvG0g=,254026,212,52.0,Robert Andrew Hyman,Biréli Lagrène,Robert Andrew Hyman


In [37]:
index = songs[songs['lyricists'] == 'Robert Andrew Hyman'].index

In [38]:
index

Int64Index([492446, 771082], dtype='int64')

In [39]:
songs = songs.drop(index)

In [40]:
songs[songs['lyricists'] == 'Robert Andrew Hyman']

,song_id,song_length,genre_ids,language,composers,artist,lyricists


# Feature Engineer Training  and Test Data

## Train Data

In [12]:
train = pd.read_csv('Data/train.csv')

In [13]:
for item in train.columns:
    print(item + ": " + str(train[item].dtype))

msno: object
song_id: object
source_system_tab: object
source_screen_name: object
source_type: object
target: int64


In [14]:
for col in train.columns:
    print(col + ": ", train[col].isnull().sum())

msno:  0
song_id:  0
source_system_tab:  24849
source_screen_name:  414804
source_type:  21539
target:  0


In [15]:
train["source_system_tab"].fillna("not_available", inplace=True)
train["source_screen_name"].fillna("not_available", inplace=True)
train["source_type"].fillna("not_available", inplace=True)

## Test Data

In [ ]:
test = pd.read_csv('Data/test.csv')

In [ ]:
for item in test.columns:
    print(item + ": " + str(test[item].dtype))

In [ ]:
for col in test.columns:
    print(col + ": ", test[col].isnull().sum())

In [ ]:
test["source_system_tab"].fillna("not_available", inplace=True)
test["source_screen_name"].fillna("not_available", inplace=True)
test["source_type"].fillna("not_available", inplace=True)

In [ ]:
test.head()

### Join Members and Sonds data to Training Data

In [49]:
train_1 = train.join(members.set_index('msno'), on='msno')

In [ ]:
train_1 = train.join(members_new.set_index('msno'), on='msno')

In [50]:
train_2 = train_1.join(songs.set_index('song_id'), on='song_id')

In [51]:
train_X = train_2.drop(['target'],axis=1)
train_y = train_2.target

In [52]:
for item in train_X.columns:
    print(item + ": " + str(train_X[item].dtype))

msno: object
song_id: object
source_system_tab: object
source_screen_name: object
source_type: object
city: int64
bd: int64
gender: object
registered_via: int64
reg_day: int64
reg_month: int64
reg_year: int64
exp_day: int64
exp_month: int64
exp_year: int64
song_length: float64
genre_ids: object
language: float64
composers: object
artist: object
lyricists: object


In [53]:
train_X.head(5)

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,reg_day,...,reg_year,exp_day,exp_month,exp_year,song_length,genre_ids,language,composers,artist,lyricists
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,0,NaN,7,2,...,2012,5,10,2017,206471.0,359,52.0,Dan Smith,Bastille,no_lyricist
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,13,24,female,9,25,...,2011,11,9,2017,284584.0,1259,52.0,no_composer,Various Artists,no_lyricist
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,13,24,female,9,25,...,2011,11,9,2017,225396.0,1259,52.0,N. Jones、W. Adams、J. Lordan、D. Ingle,Nas,no_lyricist
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,13,24,female,9,25,...,2011,11,9,2017,255512.0,1019,-1.0,Kwadwo Donkoh,Soundway,no_lyricist
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,0,NaN,7,2,...,2012,5,10,2017,187802.0,1011,52.0,Brett Young,Brett Young,no_lyricist


In [22]:
lr = LogisticRegression()
lr.fit(train_X, train_y)

ValueError: could not convert string to float: 'Robert Andrew Hyman| Cyndy Lauper'

In [34]:
lr.fit(train_X, train_y)

ValueError: could not convert string to float: 'Robert Andrew Hyman'

In [45]:
lr.fit(train_X, train_y)

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [54]:
lr.fit(train_X, train_y)

ValueError: could not convert string to float: 'online-playlist'

In [ ]:
test = test.join(members.set_index('msno'), on='msno')

In [ ]:
test = test.join(songs.set_index('song_id'), on='song_id')